# CS295/395: Secure Distributed Computation
## In-Class Exercise, Week of 9/12/2022

In [1]:
# Imports and definitions
import numpy as np
from collections import defaultdict
import galois

GF = galois.GF(97)

## Question 1

Write code to generate shares of a secret $x$ in a $(t, n)$-secret sharing scheme using Shamir's technique, where $n = 5$ and $t = 2$.

In [2]:
def shr_2_5(v):
    n = 5
    m = GF.Random()

    def f(x):
        return m * x + v

    xs = [GF(i) for i in range(1, n+1)]
    ys = [f(x) for x in xs]

    shares = list(zip(xs, ys))
    return shares

In [3]:
# Example for question 1

shr_2_5(GF(5))

[(GF(1, order=97), GF(53, order=97)),
 (GF(2, order=97), GF(4, order=97)),
 (GF(3, order=97), GF(52, order=97)),
 (GF(4, order=97), GF(3, order=97)),
 (GF(5, order=97), GF(51, order=97))]

In [4]:
# TEST CASE for question 1

assert len(shr_2_5(GF(5))) == 5

## Question 2

Write a function to reconstruct the secret, using only two shares.

In [5]:
def reconstruct(s1, s2):

    # deconstruct shares
    x1, y1 = s1
    x2, y2 = s2

    # calculate the slope
    # because we need this division we cannot use a ring it MUST be a galois field
    m = (y2 - y1) / (x2 - x1)

    # calculate the intercept
    v = y1 - m * x1

    return GF(v)

shares = shr_2_5(GF(5))
reconstruct(shares[0], shares[1])

GF(5, order=97)

In [6]:
# TEST CASE
shares = shr_2_5(GF(5))
assert reconstruct(shares[0], shares[1]) == GF(5)

## Question 3

Why is a threshold secret sharing scheme more useful than the simpler additive secret sharing scheme we saw earlier?

- You can reconstruct the secret using fewer than all of the shares
- If some parties drop out during the protocol execution, a threshold scheme still lets you reconstruct the answer
- Committee encryption/decryption (secret share an encryption key)
- Catch malicious adversaries/parties cheating

## Question 4

Write code to generate shares of a secret $x$ in a $(t, n)$-secret sharing scheme using Shamir's technique, for any $t$ and $n$.

In [7]:
def shamir_share(v, t, n):

    # generate random coefficients
    coeffs = [GF.Random() for _ in range(t-1)]
    coeffs.append(v)

    # create the polynomial
    poly = galois.Poly(coeffs, field=GF)

    # evaluate the polynomial at points 1, 2, ..., n
    shares = [(GF(i), poly(i)) for i in range(1, n+1)]

    return shares

shamir_share(GF(5), 3, 16)

[(GF(1, order=97), GF(3, order=97)),
 (GF(2, order=97), GF(77, order=97)),
 (GF(3, order=97), GF(33, order=97)),
 (GF(4, order=97), GF(65, order=97)),
 (GF(5, order=97), GF(76, order=97)),
 (GF(6, order=97), GF(66, order=97)),
 (GF(7, order=97), GF(35, order=97)),
 (GF(8, order=97), GF(80, order=97)),
 (GF(9, order=97), GF(7, order=97)),
 (GF(10, order=97), GF(10, order=97)),
 (GF(11, order=97), GF(89, order=97)),
 (GF(12, order=97), GF(50, order=97)),
 (GF(13, order=97), GF(87, order=97)),
 (GF(14, order=97), GF(6, order=97)),
 (GF(15, order=97), GF(1, order=97)),
 (GF(16, order=97), GF(72, order=97))]

In [8]:
# Example for question 1

shr_2_5(GF(5))

[(GF(1, order=97), GF(72, order=97)),
 (GF(2, order=97), GF(42, order=97)),
 (GF(3, order=97), GF(12, order=97)),
 (GF(4, order=97), GF(79, order=97)),
 (GF(5, order=97), GF(49, order=97))]

In [9]:
# TEST CASE
assert len(shamir_share(GF(5), 3, 6)) == 6
shares = shamir_share(GF(5), 2, 6)
assert reconstruct(shares[0], shares[1]) == GF(5)

## Question 5

Given the two sets of shares `shares1` and `shares2` below, write a function whose output is their sum (as a set of shares).

In [10]:
shares1 = shamir_share(GF(20), 2, 6)
shares2 = shamir_share(GF(5), 2, 6)

def add_shares(shares1, shares2):
    added_shares = []

    for s1, s2 in zip(shares1, shares2):
        x1, y1 = s1
        x2, y2 = s2

        # check that the shares are at the same point (they better be)
        assert x1 == x2

        added_shares.append((x1, y1 + y2))

    return added_shares

added_shares = add_shares(shares1, shares2)
print(added_shares)
reconstruct(added_shares[0], added_shares[1])

[(GF(1, order=97), GF(63, order=97)), (GF(2, order=97), GF(4, order=97)), (GF(3, order=97), GF(42, order=97)), (GF(4, order=97), GF(80, order=97)), (GF(5, order=97), GF(21, order=97)), (GF(6, order=97), GF(59, order=97))]


GF(25, order=97)

In [11]:
# TEST CASE
added_shares = add_shares(shares1, shares2)
assert reconstruct(added_shares[0], added_shares[2]) == GF(25)

## Question 6

Write a function to reconstruct a secret from a set of at least $t$ shares. Use the `galois.lagrange_poly` function, which implements [Lagrange interpolation](https://en.wikipedia.org/wiki/Lagrange_polynomial).

In [12]:
def reconstruct(shares):
    # create a polynomial from the shares
    xs = GF([x for x, _ in shares])
    ys = GF([y for _, y in shares])

    poly = galois.lagrange_poly(xs, ys)
    return GF(poly(0))

reconstruct(added_shares)

GF(25, order=97)

In [13]:
# TEST CASE
shares = shamir_share(GF(30), 5, 10)
assert reconstruct(shares) == GF(30)
assert reconstruct(shares[:5]) == GF(30)  # t shares are sufficient
assert reconstruct(shares[:4]) != GF(30)  # t - 1 shares are not sufficient

## Question 7

Given the two sets of shares `shares1` and `shares2` below, write a function whose output is their product (as a set of shares).

In [14]:
shares1 = shamir_share(GF(20), 3, 6)
shares2 = shamir_share(GF(3), 3, 6)

def mult_shares(shares1, shares2):
    added_shares = []

    for s1, s2 in zip(shares1, shares2):
        x1, y1 = s1
        x2, y2 = s2

        # check that the shares are at the same point (they better be)
        assert x1 == x2

        added_shares.append((x1, y1 * y2))

    return added_shares

product_shares = mult_shares(shares1, shares2)
print(product_shares)
reconstruct(shares1)
reconstruct(product_shares)

[(GF(1, order=97), GF(1, order=97)), (GF(2, order=97), GF(83, order=97)), (GF(3, order=97), GF(94, order=97)), (GF(4, order=97), GF(8, order=97)), (GF(5, order=97), GF(82, order=97)), (GF(6, order=97), GF(80, order=97))]


GF(60, order=97)

In [15]:
# TEST CASE
product_shares = mult_shares(shares1, shares2)

assert reconstruct(product_shares) == GF(60)
assert reconstruct(product_shares[:4]) != GF(60)  # t shares are no longer sufficient

## Limitation on multiplication
Every time you multiply two polynomials, the dregree increases so more shares are required to uncover the secret.

### Degree reduction
Create a new polynomial with degree d/2 with the same secret but random coeffs.

### Vandermonde matrix

## Circuits
- no conditionals (unless in binary)
- no loops
- no memory/control flow